In [42]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import time
import pandas as pd
import numpy as np
import asyncio
from requests_html import AsyncHTMLSession

MAX_CONCURRENT_CHROME = 4  # Set the maximum number of concurrent Chrome instances
semaphore = asyncio.Semaphore(MAX_CONCURRENT_CHROME)

In [43]:
# URL to scrape
base_url = "https://www.gettyimages.com/search/2/image?family=creative%2Ceditorial&phrase=vogue%20"
span_class = "JO4Dw2C5EjCB3iovKUcw"

In [46]:
async def get_npage(url, max_retries=1, delay=0.1):
    async with semaphore:
        attempt = 0
        while attempt < max_retries:
            try:
                asession = AsyncHTMLSession()
                response = await asession.get(url)
        
                # Render the JavaScript on the page (if needed)
                await response.html.arender()

                soup = BeautifulSoup(response.html.html, "html.parser")
                print(soup.prettify())


                span = soup.find('span', {'class': span_class})

                if span:
                    npages = int(span.text)
                    return (url, npages)  # return the found <div> with indentation
                else:
                    attempt += 1
                    print("Retry", url, "retries:", attempt)
                    await asession.close()
                    time.sleep(delay)
                    continue
            except Exception as e:
                print(f"Error loading {url}")
                attempt += 1
                await asession.close()
                time.sleep(delay)
            finally:
                await asession.close()

        return None


async def run_tasks(s):
    tasks = [get_npage(arg) for arg in s]
    results = await asyncio.gather(*tasks)
    return results

In [47]:
npage_list = await run_tasks(["https://www.gettyimages.com/search/2/image?family=creative%2Ceditorial&phrase=vogue%201960"])
npage_list

<!DOCTYPE html>
<html class="en-us no-js" lang="en-us" prefix="og: http://ogp.me/ns#">
 <head>
  <meta content="A/kargTFyk8MR5ueravczef/wIlTkbVk1qXQesp39nV+xNECPdLBVeYffxrM8TmZT6RArWGQVCJ0LRivD7glcAUAAACQeyJvcmlnaW4iOiJodHRwczovL2dvb2dsZS5jb206NDQzIiwiZmVhdHVyZSI6IkRpc2FibGVUaGlyZFBhcnR5U3RvcmFnZVBhcnRpdGlvbmluZzIiLCJleHBpcnkiOjE3NDIzNDIzOTksImlzU3ViZG9tYWluIjp0cnVlLCJpc1RoaXJkUGFydHkiOnRydWV9" http-equiv="origin-trial"/>
  <style type="text/css">
   @charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}
  </style>
  <script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-P4WB37" type="text/javascript">
  </script>
  <script async="" charset="utf-8" crossorigin="anonymous" integrity="sha384-gaZtYaI7oJHEOo1YO6VuoCE4iZ5fMV9mYHP0BHf+r2ttABnpWWCIkgI6ItNW4ZdO" src="https://www.gstatic.com/recaptcha

[None]